In [19]:
import pandas as pd
import numpy as np
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела (
в
е
с
в
к
г
р
о
с
т
в
м
).

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [20]:
dupl_columns = list(diabetes.columns)

mask = diabetes.duplicated(subset=dupl_columns)
d_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {d_duplicates.shape[0]}')

Число найденных дубликатов: 10


In [21]:
diabetes = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_upd.shape[0]}')

Результирующее число записей: 768


Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек)

In [22]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq >= 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio >= 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [23]:
diabetes = diabetes.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {diabetes.shape[1]}')

Результирующее число признаков: 9


Попробуйте найти пропуски в данных с помощью метода isnull().

Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.



In [24]:
diabetes['Glucose'] = diabetes['Glucose'].apply(lambda x: x if x != 0 else np.nan)
diabetes.isnull().mean()

Pregnancies                 0.00000
Glucose                     0.00651
BloodPressure               0.00000
SkinThickness               0.00000
Insulin                     0.00000
BMI                         0.00000
DiabetesPedigreeFunction    0.00000
Age                         0.00000
Outcome                     0.00000
dtype: float64

In [25]:
diabetes['BloodPressure'] = diabetes['BloodPressure'].apply(lambda x: x if x != 0 else np.nan)
diabetes['SkinThickness'] = diabetes['SkinThickness'].apply(lambda x: x if x != 0 else np.nan)
diabetes['Insulin'] = diabetes['Insulin'].apply(lambda x: x if x != 0 else np.nan)
diabetes['BMI'] = diabetes['BMI'].apply(lambda x: x if x != 0 else np.nan)
diabetes.isnull().mean()

Pregnancies                 0.000000
Glucose                     0.006510
BloodPressure               0.045573
SkinThickness               0.295573
Insulin                     0.486979
BMI                         0.014323
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
dtype: float64

Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [26]:
thresh = diabetes.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
diabetes = diabetes.dropna(thresh=thresh, axis=1)#удаляем записи, в которых есть хотя бы 1 пропуск
#отображаем результирующую долю пропусков
diabetes.isnull().mean()

Pregnancies                 0.000000
Glucose                     0.006510
BloodPressure               0.045573
SkinThickness               0.295573
BMI                         0.014323
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
dtype: float64

In [27]:
diabetes.shape[1]

8

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [28]:
#отбрасываем строки с числом пропусков более 2 в строке
m = diabetes.shape[1] #число признаков после удаления столбцов
diabetes = diabetes.dropna(thresh=m-2, axis=0)
diabetes.shape[0]

761

В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

In [29]:
values = {
    'Glucose': diabetes['Glucose'].median(),
    'BloodPressure': diabetes['BloodPressure'].median(),
    'SkinThickness': diabetes['SkinThickness'].median(),
    'BMI': diabetes['BMI'].median()
}
diabetes = diabetes.fillna(values)
print(round(diabetes['SkinThickness'].mean(), 1))

29.1


Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [35]:
def outliers_iqr(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [31]:
outliers, cleaned = outliers_iqr(diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [32]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [34]:
outliers, cleaned = outliers_z_score(diabetes, 'SkinThickness', log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.



Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

In [36]:
outliers, cleaned = outliers_iqr(diabetes, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

outliers, cleaned = outliers_iqr(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732
Число выбросов по методу Тьюки: 0
Результирующее число записей: 761


Имеются две базы данных (два листа Excel-файла): база с ценами конкурентов (Data_Parsing) и внутренняя база компании (Data_Company).

В базе парсинга есть два id, однозначно определяющие товар: producer_id и producer_color.

В базе компании есть два аналогичных поля: item_id и color_id.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: _, -, ~, \\, /.

Необходимо:

Считать данные из Excel в DataFrame (Data_Parsing) и (Data_Company).
Подтянуть к базе парсинга данные из базы компании (item_id, color_id, current_price) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).
Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.
Записать новый файл Excel с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец outlier и проставить Yes для выбросов).

In [38]:
Data_Parsing = pd.read_excel("data/Data_TSUM.xlsx", "Data_Parsing")
display(Data_Parsing)

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa1111_11,black,167
1,Valentino,Shoes,aaaaa1111_12,black,188
2,Valentino,Shoes,aaaaa1111_13,black,184
3,Valentino,Shoes,aaaaa1111_14,bla//ck,196
4,Valentino,Shoes,aaaaa1111_15,bla\\ck,250
...,...,...,...,...,...
70,Stone Island,Bags,sssss1111_31,~~~red,164
71,Stone Island,Bags,sssss1111_32,~~~red,158
72,Stone Island,Bags,sssss1111_33,~~~red,194
73,Stone Island,Bags,sssss1111_34,~~~red,256


In [60]:
Data_Parsing = Data_Parsing.replace('\\\\', '', regex=True)
display(Data_Parsing)

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa111111,black,167
1,Valentino,Shoes,aaaaa111112,black,188
2,Valentino,Shoes,aaaaa111113,black,184
3,Valentino,Shoes,aaaaa111114,black,196
4,Valentino,Shoes,aaaaa111115,black,250
...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164
71,Stone Island,Bags,sssss111132,red,158
72,Stone Island,Bags,sssss111133,red,194
73,Stone Island,Bags,sssss111134,red,256


In [39]:
Data_Company = pd.read_excel("data/Data_TSUM.xlsx", "Data_Company")
display(Data_Company)

,brand,Category,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,153
...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,165
71,Stone Island,Bags,sssss111132,red,196
72,Stone Island,Bags,sssss111133,red,236
73,Stone Island,Bags,sssss111134,red,222


In [62]:
Data_merged = Data_Parsing.merge(
    Data_Company,
    left_on=['producer_id', 'producer_color'],
    right_on=['item_id', 'color_id'],
    how="inner"
)
Data_merged = Data_merged.drop(['brand_y', 'Category_y'], axis=1)
display(Data_merged)

,brand_x,Category_x,producer_id,producer_color,price,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153
...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,sssss111131,red,165
71,Stone Island,Bags,sssss111132,red,158,sssss111132,red,196
72,Stone Island,Bags,sssss111133,red,194,sssss111133,red,236
73,Stone Island,Bags,sssss111134,red,256,sssss111134,red,222


In [71]:
Data_merged['Price_diff'] = (Data_merged['price'] - Data_merged['current price'])/Data_merged['price'] * 100

In [72]:
display(Data_merged)

,brand_x,Category_x,producer_id,producer_color,price,item_id,color_id,current price,Price_diff
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,-47.904192
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,14.361702
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,-27.173913
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,14.795918
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,38.800000
...,...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,sssss111131,red,165,-0.609756
71,Stone Island,Bags,sssss111132,red,158,sssss111132,red,196,-24.050633
72,Stone Island,Bags,sssss111133,red,194,sssss111133,red,236,-21.649485
73,Stone Island,Bags,sssss111134,red,256,sssss111134,red,222,13.281250


In [73]:
#выбросами считаем разницу цен больше чем 30%
Data_merged['Outline'] = Data_merged['Price_diff'].apply(lambda x: True if abs(x) > 30 else False)
display(Data_merged)

,brand_x,Category_x,producer_id,producer_color,price,item_id,color_id,current price,Price_diff,Outline
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,-47.904192,True
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,14.361702,False
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,-27.173913,False
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,14.795918,False
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,38.800000,True
...,...,...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,sssss111131,red,165,-0.609756,False
71,Stone Island,Bags,sssss111132,red,158,sssss111132,red,196,-24.050633,False
72,Stone Island,Bags,sssss111133,red,194,sssss111133,red,236,-21.649485,False
73,Stone Island,Bags,sssss111134,red,256,sssss111134,red,222,13.281250,False


In [76]:
with pd.ExcelWriter('data/Data_TSUM.xlsx', mode='a') as writer:
    Data_merged.to_excel(writer, 'Outline')